In [1]:
from azureml.core import Workspace, Dataset, Datastore

In [2]:
# Accessing dataset directly from Blob storage container
ws = Workspace.from_config()
datastore = Datastore.get(ws, "cvdatastore")

dataset = Dataset.File.from_files(path=(datastore, "Plant_leaf_diseases_dataset_with_augmentation/Plant_leave_diseases_dataset_with_augmentation/**"))

In [43]:
# I dont want to download the data here but directly accessing it from blob
mount_context = dataset.mount()
mount_context.start()

data_path = mount_context.mount_point
print(data_path)

/tmp/tmpz4uupnnm


In [4]:
import os

for root, dirs, files in os.walk(data_path):
    print(root, len(files))
    print(len(dirs))
    break


/tmp/tmp025igtqb 0
39


In [5]:
import torch
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torch import nn

In [45]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [48]:
full_dataset = datasets.ImageFolder(root=data_path)


train_size = int(0.8*len(full_dataset))
test_size = len(full_dataset)-train_size
train_dataset, test_dataset = random_split(full_dataset,[train_size,test_size])

In [49]:
len(full_dataset.classes)

39

In [50]:
train_dataset.dataset.transform=train_transform
test_dataset.dataset.transform = test_transform

In [51]:
trainloader = DataLoader(train_dataset,batch_size=128,shuffle=True)
testloader = DataLoader(test_dataset,batch_size=128,shuffle=False)

In [30]:
len(trainloader)

385

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [32]:
model = models.resnet18(pretrained = True)
print(model)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [33]:
# Freezing all the layers
for param in model.parameters():
    param.requires_grad=False 

# Replacing the last layer with only 39 oputputs
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 39)

model = model.to(device)

In [34]:
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [35]:
# Model training & evaluating

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    
    correct, total = 0,0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            out = model(images)
            _, pred = torch.max(out, 1)
            total += labels.size(0)
            correct += (pred==labels).sum().item()

    print(f"Epoch:{epoch+1}/{epochs}\t loss: {running_loss}\t Accuracy: {100*correct/total}%")

Epoch:1/5	 loss: 305.20731569826603	 Accuracy: 93.82013335501708%
Epoch:2/5	 loss: 90.35128190368414	 Accuracy: 95.18620913969751%
Epoch:3/5	 loss: 66.16048909723759	 Accuracy: 95.62530492763051%
Epoch:4/5	 loss: 54.370044618844986	 Accuracy: 95.89364124247845%
Epoch:5/5	 loss: 47.24691991135478	 Accuracy: 95.99934948772157%


In [54]:
mount_context.stop()